In [1113]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, average_precision_score, f1_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Загрузка и предварительная обработка данных

In [1114]:
columns = pd.read_csv('loans.txt', sep='\t', encoding='1251', nrows=0).columns

In [1115]:
columns

Index(['Код', 'Дата', 'ОД, %', 'Возраст', 'Проживание',
       'Срок проживания в регионе', 'Семейное положение', 'Образование',
       'Стаж работы на последнем месте', 'Уровень должности',
       'Кредитная история', 'Просрочки свыше 60 дн.', 'Тестовое множество'],
      dtype='object')

In [1116]:
df = pd.read_csv(
    'loans.txt', sep='\t', encoding='1251', usecols=[i for i in columns if i != 'Тестовое множество']
)

In [1117]:
df[["Код", "Просрочки свыше 60 дн."]].groupby(["Просрочки свыше 60 дн."]).count()

,Код
Просрочки свыше 60 дн.,
0,2209
1,500


In [1118]:
df.head()

,Код,Дата,"ОД, %",Возраст,Проживание,Срок проживания в регионе,Семейное положение,Образование,Стаж работы на последнем месте,Уровень должности,Кредитная история,Просрочки свыше 60 дн.
0,1,02.03.2007,51,26,Муниципальное жилье,менее 1 года,Другое,высшее,менее 1 года,сотрудник,нет данных,1
1,2,02.03.2007,46,32,Собственник,от 1 до 5 лет,Женат/замужем,высшее,свыше 3 лет,сотрудник,положительная,0
2,3,02.03.2007,13,36,Аренда,свыше 5 лет,Холост/Не замужем,среднее специальное,от 1 года до 3 лет,сотрудник,нет данных,0
3,4,02.03.2007,48,52,Муниципальное жилье,от 1 до 5 лет,Другое,высшее,менее 1 года,сотрудник,нет данных,1
4,5,02.03.2007,33,30,Собственник,от 1 до 5 лет,Холост/Не замужем,среднее специальное,менее 1 года,сотрудник,нет данных,0


In [1119]:
# Sampling
# df = df[(df["Просрочки свыше 60 дн."] == 1) | ((df["Просрочки свыше 60 дн."] == 0) & (df["Код"] < 1500))]

In [1120]:
def coder(df, old_name, new_name):
    coder_model = LabelEncoder()
    coder_model.fit(df[old_name])
    df[new_name] = coder_model.transform(df[old_name])
    

In [1121]:
coder(df, "Проживание", "Home")
coder(df, "Срок проживания в регионе", "Period")
coder(df, "Семейное положение", "Status")
coder(df, "Образование", "Degree")
coder(df, "Стаж работы на последнем месте", "WorkExperience")
coder(df, "Уровень должности", "Position")
coder(df, "Кредитная история", "History")

In [1122]:
target = "Просрочки свыше 60 дн."
features = ["ОД, %", 'Возраст', "Period", "Status", "Degree", "WorkExperience", "History", "Home"]

# Построение модели на исходных данных

In [1123]:
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], random_state=42, test_size=0.25)

In [1124]:
forest = RandomForestClassifier(random_state=42, max_depth=15, n_estimators=500, max_features=len(features))
forest.fit(X_train, y_train)
print(f"Точность на обучающей выборке: {forest.score(X_train, y_train)}")
print(f"Точность на тестовой выборке: {forest.score(X_test, y_test)}")

Точность на обучающей выборке: 0.999507631708518
Точность на тестовой выборке: 0.8643067846607669


In [1125]:
score = cross_val_score(forest, df[features], df[target], cv=5)
print(score)
print(score.mean())

[0.85424354 0.86900369 0.87822878 0.84501845 0.85582255]
0.8604634031552884


In [1126]:
pred = forest.predict(X_test)
print(confusion_matrix(y_test, pred))
print(f1_score(y_test, pred))

[[528  33]
 [ 59  58]]
0.5576923076923077


In [1127]:
average_precision_score(y_test, forest.predict_proba(X_test)[:, 1])

0.5525017174527251

In [1128]:
roc_auc_score(y_test, forest.predict_proba(X_test)[:, 1])

0.8493151728445846

In [1129]:
dtree = DecisionTreeClassifier(random_state=42, max_depth=10)
dtree.fit(X_train, y_train)
print(f"Точность на обучающей выборке: {dtree.score(X_train, y_train)}")
print(f"Точность на тестовой выборке: {dtree.score(X_test, y_test)}")

Точность на обучающей выборке: 0.9487936976858691
Точность на тестовой выборке: 0.8348082595870207


In [1130]:
pred = dtree.predict(X_test)
print(confusion_matrix(y_test, pred))

[[514  47]
 [ 65  52]]


In [1131]:
score = cross_val_score(dtree, df[features], df[target], cv=5)
print(score)
print(score.mean())

[0.80442804 0.80258303 0.84686347 0.8099631  0.83548983]
0.8198654944035578


In [1132]:
bag = BaggingClassifier(dtree, n_estimators=40, random_state=0)
bag.fit(X_train, y_train)
print(f"Точность на обучающей выборке: {bag.score(X_train, y_train)}")
print(f"Точность на тестовой выборке: {bag.score(X_test, y_test)}")

Точность на обучающей выборке: 0.9699655342195963
Точность на тестовой выборке: 0.8672566371681416


In [1133]:
pred = bag.predict(X_test)
print(confusion_matrix(y_test, pred))

[[531  30]
 [ 60  57]]


In [1134]:
score = cross_val_score(bag, df[features], df[target], cv=5)
print(score)
print(score.mean())

[0.85608856 0.86346863 0.86715867 0.84317343 0.85767098]
0.8575120557120544


# Over Sampling

In [1135]:
# Over Sampling

# df_1 = df[df["Просрочки свыше 60 дн."] == 1]

In [1136]:
# df = pd.concat(sum(([df], [df_1] * 2), []))

In [1137]:
df[["Код", "Просрочки свыше 60 дн."]].groupby(["Просрочки свыше 60 дн."]).count()

,Код
Просрочки свыше 60 дн.,
0,2209
1,500


In [1138]:
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], random_state=42, test_size=0.25)

In [1139]:
X_train_samples = X_train[X_train.index.isin(y_train[y_train == 1].index )]

In [1140]:
X_train = pd.concat(sum(([X_train], [X_train_samples] * 2), []))
y_train = pd.concat(sum(([y_train], [y_train[y_train == 1]] * 2), []))

In [1184]:
forest = RandomForestClassifier(random_state=42, max_depth=8, n_estimators=50, max_features=len(features)-1)
forest.fit(X_train, y_train)
print(f"Точность на обучающей выборке: {forest.score(X_train, y_train)}")
print(f"Точность на тестовой выборке: {forest.score(X_test, y_test)}")

Точность на обучающей выборке: 0.9163389345727565
Точность на тестовой выборке: 0.8407079646017699


In [1185]:
for i, j in zip(features, forest.feature_importances_):
    print(i, j)

ОД, % 0.1832973585597588
Возраст 0.11471191696190941
Period 0.09299189159219184
Status 0.056299011809126755
Degree 0.09318518264445444
WorkExperience 0.07566966556184823
History 0.2578612842186972
Home 0.12598368865201337


In [1186]:
pred = forest.predict(X_test)
print(confusion_matrix(y_test, pred))

[[490  71]
 [ 37  80]]


In [1187]:
score = cross_val_score(forest, df[features], df[target], cv=5)
print(score)
print(score.mean())

[0.83763838 0.87822878 0.86900369 0.84132841 0.85582255]
0.8564043625648825


In [1175]:
average_precision_score(y_test, forest.predict_proba(X_test)[:, 1])

0.5197226304981327

In [1146]:
print(f1_score(y_test, pred))

0.5954198473282442


In [1147]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [1]:
mlp = MLPClassifier(
    solver='lbfgs', random_state=0, hidden_layer_sizes=[100,10], alpha=1).fit(X_train_scaled, y_train)
print(f"Точность на обучающей выборке: {mlp.score(X_train_scaled, y_train)}")
print(f"Точность на тестовой выборке: {mlp.score(X_test_scaled, y_test)}")
pred = mlp.predict(X_test_scaled)
print(confusion_matrix(y_test, pred))
print(f1_score(y_test, pred))

NameError: name 'MLPClassifier' is not defined